In [40]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import warnings
warnings.simplefilter('ignore')

* ### Аналитическое решение

In [41]:
def analytic_solution(x,t):
    return math.cos(x+2*t)

In [42]:
def analytic_points(t):
        x1 = np.linspace(0,1,11)
        u = []
        for x in x1:
                res = analytic_solution(x,t)
                u.append(res)
        return x1,u

* ### Численное решение

In [48]:
def solve(koef,kurant):
    solutions = []; k = 1/kurant
    L0 = 11

    L = koef * (L0 - 1) + 1
    x, h = np.linspace(0, 1, L, retstep=True)
    T = int(np.ceil((1) / (kurant * h))) 
    t, tau = np.linspace(0, 1, T, retstep=True)
    u_next = lambda curr,next1,next2,next3,tau,h: curr + (tau/(3*h))*(2*next3-9*next2+18*next1-11*curr)+(2*(tau**2)/(h**2)) *(-next3+4*next2-5*next1+2*curr) + (4*tau**2)/(3*h**2) * (next3-3*next2+3*next1-curr)
    u_l0 = lambda x: math.cos(x)
    u_Ln = lambda t: math.cos(1+2*t)
    u = np.zeros([len(x),len(t)]); ux0 = []; uLn = []
    # print("L = ", L)
    # print("L = ", L, "T = ", T)
    for i in range(0,len(x)): 
            u[i][0] = u_l0(x[i])
    for j in range(1,len(t)):
            u[len(x)-1][j] = u_Ln(t[j])
    for j in range(0,len(t)-1):
        u[len(x)-2][j+1] = math.cos(1+2*t[j+1]) + h*math.sin(1+2*t[j+1])-h**2/2*math.cos(1+2*t[j+1])-(h**3)/6 * math.sin(1+2*t[j+1])
        u[len(x)-3][j+1] = math.cos(1+2*t[j+1]) + 2*h*math.sin(1+2*t[j+1])-2*(h**2)*math.cos(1+2*t[j+1])-4/3*(h**3) * math.sin(1+2*t[j+1])
        for i in range(len(x)-4,-1,-1):
            u[i][j+1] = u_next(u[i][j],u[i+1][j],u[i+2][j],u[i+3][j],tau,h)
            
    solutions = []
    for i in range(len(x)):
        solutions.append(u[i][len(t)-1])
    # print("h = {}".format(h), "tau = {}".format(tau), "Число куранта = {}".format(tau/h))
    return solutions[::koef]

* ### Таблица результатов

In [60]:
x,u1 = analytic_points(1)
koef = 4
u2 = solve(koef,0.5)
delta1 = []; 
for i in range(len(u1)):
    delta1.append(abs(u1[i] - u2[i]))
pd.options.display.float_format = "{:.4e}".format
print("Максимальная норма ошибки =", max(delta1))
data = pd.DataFrame({"x": x, "u аналит.": u1, "u числ. ": u2 ,"u числ.- u аналит.": delta1})
display(data)

Максимальная норма ошибки = 8.06716726744483e-05


,x,u аналит.,u числ.,u числ.- u аналит.
0,0.0000e+00,-4.1615e-01,-4.1607e-01,8.0672e-05
1,1.0000e-01,-5.0485e-01,-5.0477e-01,7.8084e-05
2,2.0000e-01,-5.8850e-01,-5.8844e-01,6.3007e-05
3,3.0000e-01,-6.6628e-01,-6.6623e-01,5.0160e-05
4,4.0000e-01,-7.3739e-01,-7.3736e-01,3.8404e-05
5,5.0000e-01,-8.0114e-01,-8.0112e-01,2.7829e-05
6,6.0000e-01,-8.5689e-01,-8.5687e-01,1.8658e-05
7,7.0000e-01,-9.0407e-01,-9.0406e-01,1.1096e-05
8,8.0000e-01,-9.4222e-01,-9.4222e-01,5.3139e-06
9,9.0000e-01,-9.7096e-01,-9.7096e-01,1.4410e-06


4.233141506288646e-05

* ### Исследование на устойчивость

In [49]:
x,u1 = analytic_points(1)
i = 0; kurant = [0.05];
kurant = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]

In [50]:
L = [1201 for i in range(len(kurant))]; deltas = []
for k in kurant:
    delta1 = []
    u2 = solve(120,k)
    for i in range(len(u1)):
        delta1.append(abs(u1[i] - u2[i]))
    deltas.append(max(delta1))
# pd.options.display.float_format = "{:.4e}".format
data = pd.DataFrame({"L": L, "Число Куранта": kurant,"Ошибка": deltas})
display(data)

,L,Число Куранта,Ошибка
0,1201,0.05,4.552230e+17
1,1201,0.10,9.328449e+32
2,1201,0.20,1.236071e+81
3,1201,0.30,1.360524e+186
4,1201,0.40,NaN
5,1201,0.50,NaN
6,1201,0.60,NaN
7,1201,0.70,NaN
8,1201,0.75,2.798138e+216
9,1201,0.80,6.434306e+64


In [ ]:
# kurant = [ 1.05, 1.1, 1.5] 
# L = [1281 for i in range(len(kurant))]; deltas = []
# for k in kurant:
#     delta1 = []
#     u2 = solve(10,k)
#     for i in range(len(u1)):
#         delta1.append(abs(u1[i] - u2[i]))
#     deltas.append(max(delta1))
# # pd.options.display.float_format = "{:.4e}".format
# data = pd.DataFrame({"L": L, "Число Куранта": kurant,"Ошибка": deltas})
# display(data)